In [68]:
import ipywidgets as widgets
from IPython.display import display

# GUI elements

api_header = widgets.HBox([widgets.Label(value="Enter Genius API Key")])

api_text = widgets.Text(
    value='',
    placeholder='Enter Genius API Key..',
    description='API:',
    disabled=False
)

api_button = widgets.Button(
    description='Store API',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Store API Key',
    icon='save' # (FontAwesome names without the `fa-` prefix)
)

canvas_header = widgets.HBox([widgets.Label(value="Canvas Settings")])

height_text = widgets.Text(
    value='',
    placeholder='Enter canvas height..',
    description='Height:',
    disabled=False
)

width_text = widgets.Text(
    value='',
    placeholder='Enter canvas width..',
    description='Width:',
    disabled=False
)

font_size_text = widgets.Text(
    values='',
    placeholder='Enter font size..',
    description='Font Size:',
    disabled=False
)

save_canvas_button = widgets.Button(
    description='Save Canvas',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save Settings for Canvas',
    icon='save' # (FontAwesome names without the `fa-` prefix)
)

header = widgets.HBox([widgets.Label(value="Sound Cloud Search")])

song_text = widgets.Text(
    value='',
    placeholder='Enter song name..',
    description='Song:',
    disabled=False
)

artist_text = widgets.Text(
    value='',
    placeholder='Enter artist name..',
    description='Artist:',
    disabled=False
)

generate_button = widgets.Button(
    description='Generate Cloud',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Generate Word Cloud',
    icon='cloud' # (FontAwesome names without the `fa-` prefix)
)

clear_button = widgets.Button(
    description='Clear Search',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear Search',
    icon='times-circle' # (FontAwesome names without the `fa-` prefix)
)

output = widgets.Output()

In [69]:
# lyricsgenius

import lyricsgenius

genius = lyricsgenius.Genius(api_text.value)

In [70]:
# Button handling
from collections import Counter
from itertools import dropwhile

lyric_count = Counter()
canvas_height = 0
canvas_width = 0
font_size = ""

def api_button_clicked(b):
    genius = lyricsgenius.Genius(api_text.value)
    with output:
        print("[DEBUG - api_button_clicked]\nAPI key:", api_text.value, "\n")
        
def save_canvas_button_clicked(b):
    global canvas_height
    global canvas_width
    global font_size
    if(int(height_text.value) < 100 or int(width_text.value) < 100):
        with output:
            print("[DEBUG - save_canvas_button_clicked]\nValues are too small (minimum 100 x 100)\n")
    else:
        canvas_height = int(height_text.value)
        canvas_width = int(width_text.value)
        font_size = font_size_text.value
        with output:
            print("[DEBUG - save_canvas_button_clicked]\nHeight:", height_text.value, "\nWidth:", width_text.value, "\nFont Size:", font_size_text.value, "\n")

def generate_button_clicked(b):
    with output:
        print("[DEBUG - generate_button_clicked]\nSong:", song_text.value, "\nArtist:", artist_text.value, "\n")
        try:
            song = genius.search_song(song_text.value, artist_text.value)

            lyrics = song.lyrics.split()
            lyrics = [x.lower() for x in lyrics] # converts all values in list to lowercase
            lyrics = [x.replace(',','') for x in lyrics] # removes all commas            
            global lyric_count
            lyric_count = Counter(lyrics)

            # Remove lyrics where frequency is >= 4, needs to be dynamic based on song
            for key, count in dropwhile(lambda key_count: key_count[1] >= 4, lyric_count.most_common()):
                del lyric_count[key]
                
            print("[DEBUG - getting lyrics]\n",lyric_count,"\n")
        except TypeError as err:
            print("\n\t--- ERROR - Enter API key correctly ---\t\n")
        
def clear_button_clicked(b):
    song_text.value = ''
    artist_text.value = ''
    height_text.value= ''
    width_text.value = ''
    with output:
        print("[DEBUG - clear_clicked()]\nCleared 'song_text', 'artist_text', 'height_text', and 'width_text'\n")

api_button.on_click(api_button_clicked)
save_canvas_button.on_click(save_canvas_button_clicked)
generate_button.on_click(generate_button_clicked)
clear_button.on_click(clear_button_clicked)

In [71]:
# Display GUI
display(api_header, api_text, api_button, canvas_header, height_text, width_text, font_size_text, save_canvas_button, header, song_text, artist_text, generate_button, clear_button, output)

Text(value='', description='API:', placeholder='Enter Genius API Key..')

Button(button_style='info', description='Store API', icon='save', style=ButtonStyle(), tooltip='Store API Key'…

Text(value='', description='Height:', placeholder='Enter canvas height..')

Text(value='', description='Width:', placeholder='Enter canvas width..')

Text(value='', description='Font Size:', placeholder='Enter font size..')

...

Text(value='', description='Song:', placeholder='Enter song name..')

Text(value='', description='Artist:', placeholder='Enter artist name..')

Button(button_style='success', description='Generate Cloud', icon='cloud', style=ButtonStyle(), tooltip='Gener…

Button(button_style='danger', description='Clear Search', icon='times-circle', style=ButtonStyle(), tooltip='C…

Output()

In [72]:
# Canvas

from ipycanvas import Canvas
import random

# creating canvas
canvas = Canvas(width=canvas_width, height=canvas_height)

# setting font size
canvas.font = font_size + 'px serif'

# stroke_text or fill_text(text, x, y, max_width=None)
for i in enumerate(lyric_count):
    canvas.fill_text(i[1], random.randint(int(canvas_width * 0.125), int(canvas_height - (canvas_height * 0.125))), random.randint(int(canvas_width * 0.125), int(canvas_height - (canvas_height * 0.125))))

# call canvas
canvas

100.0 700.0


Canvas(height=800, width=800)